In [2]:
import sqlite3
import pandas as pd
import re
from unicodedata import normalize

In [3]:
def db_con() -> sqlite3.Connection:
    path = "../ffgcrawl/db.sqlite"
    con: sqlite3.Connection = sqlite3.connect(path)
    con.row_factory = sqlite3.Row
    return con

In [4]:
def query(sql: str) -> pd.DataFrame:
    return pd.read_sql(sql, db_con())

In [5]:
texts = query("select * from texts")

In [6]:
t = texts.content[0]

In [7]:
def clean_text(t: str) -> str:
    return normalize("NFKD", re.sub(r"\s+", "", t))

In [8]:
contents = texts.content.map(clean_text)

In [10]:
texts

,url,title,content,created_at
0,https://www.fukuoka-fg.com/,ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:26.732406
1,https://www.fukuoka-fg.com/,ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.568761
2,https://www.fukuoka-fg.com/investor/public_not...,電子公告 | ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.676966
3,https://www.fukuoka-fg.com/company/conflicts.html,利益相反管理方針 | ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.679578
4,https://www.fukuoka-fg.com/company/customer_pr...,顧客保護等管理方針(FFG) | ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.685454
5,https://www.fukuoka-fg.com/company/privacy/,個人情報保護方針 | ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.688076
6,https://www.fukuoka-fg.com/policy.html,本サイトのご利用にあたって | ふくおかフィナンシャルグループ,\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自由な...,2023-04-04T16:33:30.691629
7,https://www.fukuoka-fg.com/external.html?targe...,ふくおかフィナンシャルグループ,\r\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自...,2023-04-04T16:33:30.693954
8,https://www.fukuoka-fg.com/external.html?targe...,ふくおかフィナンシャルグループ,\r\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自...,2023-04-04T16:33:30.695280
9,https://www.fukuoka-fg.com/external.html?targe...,ふくおかフィナンシャルグループ,\r\n\r\n\r\nふくおかフィナンシャルグループ\r\n\r\n\r\nおからだの不自...,2023-04-04T16:33:30.696549


In [9]:
with open("texts.txt", "w") as fp:
    for c in contents:
        fp.write(c + "\n")
        fp.write("===========\n")